In [1]:
!pip install -q sklearn

In [2]:
from __future__ import absolute_import, division, print_function, unicode_literals

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from IPython.display import clear_output
from six.moves import urllib

import tensorflow.compat.v2.feature_column as fc

import tensorflow as tf

In [17]:
dftrain = pd.read_csv(r'E:\DATASET\YoutubeDataBackUp\youtube_rebuffer_features.csv') 
dfeval = pd.read_csv(r'E:\DATASET\YoutubeDataBackUp\youtube_rebuffer_features.csv')

y_train = dftrain.pop('rebufferFlag')
y_eval = dfeval.pop('rebufferFlag')

In [18]:
dftrain.head()

,serverSynFlags,serverFinFlags,serverAckFlags,serverUrgFlags,serverRstFlags,serverPshFlags,serverByteCount,serverPacketCount,serverMinBytesPerPacket,serverMaxBytesPerPacket,...,lastuserKurRetransmit,lastuserSkeRetransmit,lastuserGoodput,lastuserMinRTT,lastuserMaxRTT,lastuserMedRTT,lastuserAvgRTT,lastuserStdRTT,lastuserKurRTT,lastuserSkeRTT
0,0,1,1062,0,0,68,1538978,1062,0,1460,...,82.136132,8.919883,9979.0,0.034809,3.790140,0.245094,0.554985,0.751516,4.964011,2.300835
1,0,0,1586,0,0,126,2283946,1586,0,1460,...,2.166667,2.041241,6755.0,0.201941,0.201941,0.201941,0.201941,0.000000,-3.000000,0.000000
2,0,1,1,0,0,1,60,1,60,60,...,1.200000,1.788854,11080.0,0.099897,0.205040,0.164032,0.158250,0.044322,-1.753397,-0.165891
3,0,0,1058,0,0,85,1524266,1058,0,1460,...,-3.000000,0.000000,11220.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
4,0,0,160,0,0,16,225279,160,0,1460,...,-1.500000,0.707107,5850.0,0.128984,0.216961,0.172973,0.172973,0.043988,-2.000000,0.000000


In [19]:
dftrain.describe()

,serverSynFlags,serverFinFlags,serverAckFlags,serverUrgFlags,serverRstFlags,serverPshFlags,serverByteCount,serverPacketCount,serverMinBytesPerPacket,serverMaxBytesPerPacket,...,lastuserKurRetransmit,lastuserSkeRetransmit,lastuserGoodput,lastuserMinRTT,lastuserMaxRTT,lastuserMedRTT,lastuserAvgRTT,lastuserStdRTT,lastuserKurRTT,lastuserSkeRTT
count,78973.000000,78973.000000,78973.000000,78973.0,78973.000000,78973.000000,7.897300e+04,78973.000000,78973.000000,78973.000000,...,78973.000000,78973.000000,78973.000000,78973.000000,78973.000000,78973.000000,78973.000000,78973.000000,78973.000000,78973.000000
mean,0.224431,0.164183,752.052942,0.0,0.000013,53.247173,1.087770e+06,752.052955,77.619148,1108.337989,...,1.409181,0.839795,7562.104044,0.136496,0.377829,0.169639,0.185090,0.061885,-0.803205,0.189875
std,1.222604,0.557253,726.621472,0.0,0.003558,58.018814,1.053481e+06,726.621459,312.576744,618.714199,...,14.182143,1.596619,5398.259455,0.338871,2.556383,0.417365,0.433975,0.425579,5.033957,0.765398
min,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000e+00,0.000000,0.000000,0.000000,...,-3.000000,-1.154701,-4170.000000,0.000000,0.000000,0.000000,0.000000,0.000000,-3.000000,-1.780898
25%,0.000000,0.000000,3.000000,0.0,0.000000,0.000000,1.460000e+03,3.000000,0.000000,1460.000000,...,-2.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,-3.000000,0.000000
50%,0.000000,0.000000,751.000000,0.0,0.000000,43.000000,1.085779e+06,751.000000,0.000000,1460.000000,...,-1.500000,0.000000,10396.666670,0.130892,0.200033,0.171185,0.180721,0.000000,-1.500000,0.000000
75%,0.000000,0.000000,1301.000000,0.0,0.000000,89.000000,1.883336e+06,1301.000000,0.000000,1460.000000,...,0.000000,1.154701,11220.000000,0.201941,0.309944,0.236034,0.256062,0.041142,0.000000,0.000000
max,32.000000,10.000000,5517.000000,0.0,1.000000,843.000000,7.975412e+06,5517.000000,1460.000000,1460.000000,...,148.816068,11.961576,61230.000000,57.161093,315.184116,57.161093,57.161093,53.547502,134.748970,11.688443


In [20]:
dftrain.shape

(78973, 230)

In [21]:
y_train.head()

0    0
1    0
2    0
3    0
4    0
Name: rebufferFlag, dtype: int64

In [22]:
#REBUFFER
feature_columns=['serverSynFlags', 'serverFinFlags', 'serverAckFlags', 'serverUrgFlags', 'serverRstFlags', 'serverPshFlags', 'serverByteCount', 'serverPacketCount', 'serverMinBytesPerPacket', 'serverMaxBytesPerPacket', 'serverMedBytesPerPacket', 'serverAvgBytesPerPacket', 'serverStdBytesPerPacket', 'serverKurBytesPerPacket', 'serverSkeBytesPerPacket', 'serverStrBytesPerPacket', 'serverEndBytesPerPacket', 'serverMinBytesInFlight', 'serverMaxBytesInFlight', 'serverMedBytesInFlight', 'serverAvgBytesInFlight', 'serverStdBytesInFlight', 'serverKurBytesInFlight', 'serverSkeBytesInFlight', 'serverMinInterArrivalTime', 'serverMaxInterArrivalTime', 'serverMedInterArrivalTime', 'serverAvgInterArrivalTime', 'serverStdInterArrivalTime', 'serverKurInterArrivalTime', 'serverSkeInterArrivalTime', 'serverMinRwnd', 'serverMaxRwnd', 'serverMedRwnd', 'serverAvgRwnd', 'serverStdRwnd', 'serverKurRwnd', 'serverSkeRwnd', 'serverThroughput', 'serverIdleTime', 'serverOutOfOrderBytes', 'serverOutOfOrderPackets', 'serverZeroRetransmit', 'serverOneRetransmit', 'serverTwoRetransmit', 'serverXRetransmit', 'serverMinRetransmit', 'serverMaxRetransmit', 'serverMedRetransmit', 'serverAvgRetransmit', 'serverStdRetransmit', 'serverKurRetransmit', 'serverSkeRetransmit', 'serverGoodput', 'userSynFlags', 'userFinFlags', 'userAckFlags', 'userUrgFlags', 'userRstFlags', 'userPshFlags', 'userByteCount', 'userPacketCount', 'userMinBytesPerPacket', 'userMaxBytesPerPacket', 'userMedBytesPerPacket', 'userAvgBytesPerPacket', 'userStdBytesPerPacket', 'userKurBytesPerPacket', 'userSkeBytesPerPacket', 'userStrBytesInFlight', 'userEndBytesInFlight', 'userMinBytesInFlight', 'userMaxBytesInFlight', 'userMedBytesInFlight', 'userAvgBytesInFlight', 'userStdBytesInFlight', 'userKurBytesInFlight', 'userSkeBytesInFlight', 'userMinInterArrivalTime', 'userMaxInterArrivalTime', 'userMedInterArrivalTime', 'userAvgInterArrivalTime', 'userStdInterArrivalTime', 'userKurInterArrivalTime', 'userSkeInterArrivalTime', 'userMinRwnd', 'userMaxRwnd', 'userMedRwnd', 'userAvgRwnd', 'userStdRwnd', 'userKurRwnd', 'userSkeRwnd', 'userThroughput', 'userIdleTime', 'userOutOfOrderBytes', 'userOutOfOrderPackets', 'userZeroRetransmit', 'userOneRetransmit', 'userTwoRetransmit', 'userXRetransmit', 'userMinRetransmit', 'userMaxRetransmit', 'userMedRetransmit', 'userAvgRetransmit', 'userStdRetransmit', 'userKurRetransmit', 'userSkeRetransmit', 'userGoodput', 'userMinRTT', 'userMaxRTT', 'userMedRTT', 'userAvgRTT', 'userStdRTT', 'userKurRTT', 'userSkeRTT', 'lastserverSynFlags', 'lastserverFinFlags', 'lastserverAckFlags', 'lastserverUrgFlags', 'lastserverRstFlags', 'lastserverPshFlags', 'lastserverByteCount', 'lastserverPacketCount', 'lastserverMinBytesPerPacket', 'lastserverMaxBytesPerPacket', 'lastserverMedBytesPerPacket', 'lastserverAvgBytesPerPacket', 'lastserverStdBytesPerPacket', 'lastserverKurBytesPerPacket', 'lastserverSkeBytesPerPacket', 'lastserverStrBytesInFlight', 'lastserverEndBytesInFlight', 'lastserverMinBytesInFlight', 'lastserverMaxBytesInFlight', 'lastserverMedBytesInFlight', 'lastserverAvgBytesInFlight', 'lastserverStdBytesInFlight', 'lastserverKurBytesInFlight', 'lastserverSkeBytesInFlight', 'lastserverMinInterArrivalTime', 'lastserverMaxInterArrivalTime', 'lastserverMedInterArrivalTime', 'lastserverAvgInterArrivalTime', 'lastserverStdInterArrivalTime', 'lastserverKurInterArrivalTime', 'lastserverSkeInterArrivalTime', 'lastserverMinRwnd', 'lastserverMaxRwnd', 'lastserverMedRwnd', 'lastserverAvgRwnd', 'lastserverStdRwnd', 'lastserverKurRwnd', 'lastserverSkeRwnd', 'lastserverThroughput', 'lastserverIdleTime', 'lastserverOutOfOrderBytes', 'lastserverOutOfOrderPackets', 'lastserverZeroRetransmit', 'lastserverOneRetransmit', 'lastserverTwoRetransmit', 'lastserverXRetransmit', 'lastserverMinRetransmit', 'lastserverMaxRetransmit', 'lastserverMedRetransmit', 'lastserverAvgRetransmit', 'lastserverStdRetransmit', 'lastserverKurRetransmit', 'lastserverSkeRetransmit', 'lastserverGoodput', 'lastuserSynFlags', 'lastuserFinFlags', 'lastuserAckFlags', 'lastuserUrgFlags', 'lastuserRstFlags', 'lastuserPshFlags', 'lastuserByteCount', 'lastuserPacketCount', 'lastuserMinBytesPerPacket', 'lastuserMaxBytesPerPacket', 'lastuserMedBytesPerPacket', 'lastuserAvgBytesPerPacket', 'lastuserStdBytesPerPacket', 'lastuserKurBytesPerPacket', 'lastuserSkeBytesPerPacket', 'lastuserStrBytesInFlight', 'lastuserEndBytesInFlight', 'lastuserMinBytesInFlight', 'lastuserMaxBytesInFlight', 'lastuserMedBytesInFlight', 'lastuserAvgBytesInFlight', 'lastuserStdBytesInFlight', 'lastuserKurBytesInFlight', 'lastuserSkeBytesInFlight', 'lastuserMinInterArrivalTime', 'lastuserMaxInterArrivalTime', 'lastuserMedInterArrivalTime', 'lastuserAvgInterArrivalTime', 'lastuserStdInterArrivalTime', 'lastuserKurInterArrivalTime', 'lastuserSkeInterArrivalTime', 'lastuserMinRwnd', 'lastuserMaxRwnd', 'lastuserMedRwnd', 'lastuserAvgRwnd', 'lastuserStdRwnd', 'lastuserKurRwnd', 'lastuserSkeRwnd', 'lastuserThroughput', 'lastuserIdleTime', 'lastuserOutOfOrderBytes', 'lastuserOutOfOrderPackets', 'lastuserZeroRetransmit', 'lastuserOneRetransmit', 'lastuserTwoRetransmit', 'lastuserXRetransmit', 'lastuserMinRetransmit', 'lastuserMaxRetransmit', 'lastuserMedRetransmit', 'lastuserAvgRetransmit', 'lastuserStdRetransmit', 'lastuserKurRetransmit', 'lastuserSkeRetransmit', 'lastuserGoodput', 'lastuserMinRTT', 'lastuserMaxRTT', 'lastuserMedRTT', 'lastuserAvgRTT', 'lastuserStdRTT', 'lastuserKurRTT', 'lastuserSkeRTT']

#quality
#feature_columns=['serverSynFlags', 'serverFinFlags', 'serverAckFlags', 'serverUrgFlags', 'serverRstFlags', 'serverPshFlags', 'serverByteCount', 'serverPacketCount', 'serverMinBytesPerPacket', 'serverMaxBytesPerPacket', 'serverMedBytesPerPacket', 'serverAvgBytesPerPacket', 'serverStdBytesPerPacket', 'serverKurBytesPerPacket', 'serverSkeBytesPerPacket', 'serverStrBytesPerPacket', 'serverEndBytesPerPacket', 'serverMinBytesInFlight', 'serverMaxBytesInFlight', 'serverMedBytesInFlight', 'serverAvgBytesInFlight', 'serverStdBytesInFlight', 'serverKurBytesInFlight', 'serverSkeBytesInFlight', 'serverMinInterArrivalTime', 'serverMaxInterArrivalTime', 'serverMedInterArrivalTime', 'serverAvgInterArrivalTime', 'serverStdInterArrivalTime', 'serverKurInterArrivalTime', 'serverSkeInterArrivalTime', 'serverMinRwnd', 'serverMaxRwnd', 'serverMedRwnd', 'serverAvgRwnd', 'serverStdRwnd', 'serverKurRwnd', 'serverSkeRwnd', 'serverThroughput', 'serverIdleTime', 'serverOutOfOrderBytes', 'serverOutOfOrderPackets', 'serverZeroRetransmit', 'serverOneRetransmit', 'serverTwoRetransmit', 'serverXRetransmit', 'serverMinRetransmit', 'serverMaxRetransmit', 'serverMedRetransmit', 'serverAvgRetransmit', 'serverStdRetransmit', 'serverKurRetransmit', 'serverSkeRetransmit', 'serverGoodput', 'serverBitrateChange', 'userSynFlags', 'userFinFlags', 'userAckFlags', 'userUrgFlags', 'userRstFlags', 'userPshFlags', 'userByteCount', 'userPacketCount', 'userMinBytesPerPacket', 'userMaxBytesPerPacket', 'userMedBytesPerPacket', 'userAvgBytesPerPacket', 'userStdBytesPerPacket', 'userKurBytesPerPacket', 'userSkeBytesPerPacket', 'userStrBytesInFlight', 'userEndBytesInFlight', 'userMinBytesInFlight', 'userMaxBytesInFlight', 'userMedBytesInFlight', 'userAvgBytesInFlight', 'userStdBytesInFlight', 'userKurBytesInFlight', 'userSkeBytesInFlight', 'userMinInterArrivalTime', 'userMaxInterArrivalTime', 'userMedInterArrivalTime', 'userAvgInterArrivalTime', 'userStdInterArrivalTime', 'userKurInterArrivalTime', 'userSkeInterArrivalTime', 'userMinRwnd', 'userMaxRwnd', 'userMedRwnd', 'userAvgRwnd', 'userStdRwnd', 'userKurRwnd', 'userSkeRwnd', 'userThroughput', 'userIdleTime', 'userOutOfOrderBytes', 'userOutOfOrderPackets', 'userZeroRetransmit', 'userOneRetransmit', 'userTwoRetransmit', 'userXRetransmit', 'userMinRetransmit', 'userMaxRetransmit', 'userMedRetransmit', 'userAvgRetransmit', 'userStdRetransmit', 'userKurRetransmit', 'userSkeRetransmit', 'userGoodput', 'userMinRTT', 'userMaxRTT', 'userMedRTT', 'userAvgRTT', 'userStdRTT', 'userKurRTT', 'userSkeRTT', 'lastserverSynFlags', 'lastserverFinFlags', 'lastserverAckFlags', 'lastserverUrgFlags', 'lastserverRstFlags', 'lastserverPshFlags', 'lastserverByteCount', 'lastserverPacketCount', 'lastserverMinBytesPerPacket', 'lastserverMaxBytesPerPacket', 'lastserverMedBytesPerPacket', 'lastserverAvgBytesPerPacket', 'lastserverStdBytesPerPacket', 'lastserverKurBytesPerPacket', 'lastserverSkeBytesPerPacket', 'lastserverStrBytesInFlight', 'lastserverEndBytesInFlight', 'lastserverMinBytesInFlight', 'lastserverMaxBytesInFlight', 'lastserverMedBytesInFlight', 'lastserverAvgBytesInFlight', 'lastserverStdBytesInFlight', 'lastserverKurBytesInFlight', 'lastserverSkeBytesInFlight', 'lastserverMinInterArrivalTime', 'lastserverMaxInterArrivalTime', 'lastserverMedInterArrivalTime', 'lastserverAvgInterArrivalTime', 'lastserverStdInterArrivalTime', 'lastserverKurInterArrivalTime', 'lastserverSkeInterArrivalTime', 'lastserverMinRwnd', 'lastserverMaxRwnd', 'lastserverMedRwnd', 'lastserverAvgRwnd', 'lastserverStdRwnd', 'lastserverKurRwnd', 'lastserverSkeRwnd', 'lastserverThroughput', 'lastserverIdleTime', 'lastserverOutOfOrderBytes', 'lastserverOutOfOrderPackets', 'lastserverZeroRetransmit', 'lastserverOneRetransmit', 'lastserverTwoRetransmit', 'lastserverXRetransmit', 'lastserverMinRetransmit', 'lastserverMaxRetransmit', 'lastserverMedRetransmit', 'lastserverAvgRetransmit', 'lastserverStdRetransmit', 'lastserverKurRetransmit', 'lastserverSkeRetransmit', 'lastserverGoodput', 'lastserverBitrateChange', 'lastuserSynFlags', 'lastuserFinFlags', 'lastuserAckFlags', 'lastuserUrgFlags', 'lastuserRstFlags', 'lastuserPshFlags', 'lastuserByteCount', 'lastuserPacketCount', 'lastuserMinBytesPerPacket', 'lastuserMaxBytesPerPacket', 'lastuserMedBytesPerPacket', 'lastuserAvgBytesPerPacket', 'lastuserStdBytesPerPacket', 'lastuserKurBytesPerPacket', 'lastuserSkeBytesPerPacket', 'lastuserStrBytesInFlight', 'lastuserEndBytesInFlight', 'lastuserMinBytesInFlight', 'lastuserMaxBytesInFlight', 'lastuserMedBytesInFlight', 'lastuserAvgBytesInFlight', 'lastuserStdBytesInFlight', 'lastuserKurBytesInFlight', 'lastuserSkeBytesInFlight', 'lastuserMinInterArrivalTime', 'lastuserMaxInterArrivalTime', 'lastuserMedInterArrivalTime', 'lastuserAvgInterArrivalTime', 'lastuserStdInterArrivalTime', 'lastuserKurInterArrivalTime', 'lastuserSkeInterArrivalTime', 'lastuserMinRwnd', 'lastuserMaxRwnd', 'lastuserMedRwnd', 'lastuserAvgRwnd', 'lastuserStdRwnd', 'lastuserKurRwnd', 'lastuserSkeRwnd', 'lastuserThroughput', 'lastuserIdleTime', 'lastuserOutOfOrderBytes', 'lastuserOutOfOrderPackets', 'lastuserZeroRetransmit', 'lastuserOneRetransmit', 'lastuserTwoRetransmit', 'lastuserXRetransmit', 'lastuserMinRetransmit', 'lastuserMaxRetransmit', 'lastuserMedRetransmit', 'lastuserAvgRetransmit', 'lastuserStdRetransmit', 'lastuserKurRetransmit', 'lastuserSkeRetransmit', 'lastuserGoodput', 'lastuserMinRTT', 'lastuserMaxRTT', 'lastuserMedRTT', 'lastuserAvgRTT', 'lastuserStdRTT', 'lastuserKurRTT', 'lastuserSkeRTT', 'protocol']

#rebufferquic
#feature_columns=['serverByteCount', 'serverPacketCount', 'serverMinBytesPerPacket', 'serverMaxBytesPerPacket', 'serverMedBytesPerPacket', 'serverAvgBytesPerPacket', 'serverStdBytesPerPacket', 'serverKurBytesPerPacket', 'serverSkeBytesPerPacket', 'serverMinInterArrivalTime', 'serverMaxInterArrivalTime', 'serverMedInterArrivalTime', 'serverAvgInterArrivalTime', 'serverStdInterArrivalTime', 'serverKurInterArrivalTime', 'serverSkeInterArrivalTime', 'serverThroughput', 'serverIdleTime', 'serverBitrateChange', 'userByteCount', 'userPacketCount', 'userMinBytesPerPacket', 'userMaxBytesPerPacket', 'userMedBytesPerPacket', 'userAvgBytesPerPacket', 'userStdBytesPerPacket', 'userKurBytesPerPacket', 'userSkeBytesPerPacket', 'userMinInterArrivalTime', 'userMaxInterArrivalTime', 'userMedInterArrivalTime', 'userAvgInterArrivalTime', 'userStdInterArrivalTime', 'userKurInterArrivalTime', 'userSkeInterArrivalTime', 'userThroughput', 'userIdleTime', 'lastserverByteCount', 'lastserverPacketCount', 'lastserverMinBytesPerPacket', 'lastserverMaxBytesPerPacket', 'lastserverMedBytesPerPacket', 'lastserverAvgBytesPerPacket', 'lastserverStdBytesPerPacket', 'lastserverKurBytesPerPacket', 'lastserverSkeBytesPerPacket', 'lastserverMinInterArrivalTime', 'lastserverMaxInterArrivalTime', 'lastserverMedInterArrivalTime', 'lastserverAvgInterArrivalTime', 'lastserverStdInterArrivalTime', 'lastserverKurInterArrivalTime', 'lastserverSkeInterArrivalTime', 'lastserverThroughput', 'lastserverIdleTime', 'lastserverBitrateChange', 'lastuserByteCount', 'lastuserPacketCount', 'lastuserMinBytesPerPacket', 'lastuserMaxBytesPerPacket', 'lastuserMedBytesPerPacket', 'lastuserAvgBytesPerPacket', 'lastuserStdBytesPerPacket', 'lastuserKurBytesPerPacket', 'lastuserSkeBytesPerPacket', 'lastuserMinInterArrivalTime', 'lastuserMaxInterArrivalTime', 'lastuserMedInterArrivalTime', 'lastuserAvgInterArrivalTime', 'lastuserStdInterArrivalTime', 'lastuserKurInterArrivalTime', 'lastuserSkeInterArrivalTime', 'lastuserThroughput', 'lastuserIdleTime']

#qualityquic
feature_columns=['video', 'serverSynFlags', 'serverFinFlags', 'serverAckFlags', 'serverUrgFlags', 'serverRstFlags', 'serverPshFlags', 'serverByteCount', 'serverPacketCount', 'serverMinBytesPerPacket', 'serverMaxBytesPerPacket', 'serverMedBytesPerPacket', 'serverAvgBytesPerPacket', 'serverStdBytesPerPacket', 'serverKurBytesPerPacket', 'serverSkeBytesPerPacket', 'serverMinBytesInFlight', 'serverMaxBytesInFlight', 'serverMedBytesInFlight', 'serverAvgBytesInFlight', 'serverStdBytesInFlight', 'serverKurBytesInFlight', 'serverSkeBytesInFlight', 'serverMinInterArrivalTime', 'serverMaxInterArrivalTime', 'serverMedInterArrivalTime', 'serverAvgInterArrivalTime', 'serverStdInterArrivalTime', 'serverKurInterArrivalTime', 'serverSkeInterArrivalTime', 'serverMinRwnd', 'serverMaxRwnd', 'serverMedRwnd', 'serverAvgRwnd', 'serverStdRwnd', 'serverKurRwnd', 'serverSkeRwnd', 'serverThroughput', 'serverIdleTime', 'serverOutOfOrderBytes', 'serverOutOfOrderPackets', 'serverZeroRetransmit', 'serverOneRetransmit', 'serverTwoRetransmit', 'serverXRetransmit', 'serverMinRetransmit', 'serverMaxRetransmit', 'serverMedRetransmit', 'serverAvgRetransmit', 'serverStdRetransmit', 'serverKurRetransmit', 'serverSkeRetransmit', 'serverGoodput', 'userSynFlags', 'userFinFlags', 'userAckFlags', 'userUrgFlags', 'userRstFlags', 'userPshFlags', 'userByteCount', 'userPacketCount', 'userMinBytesPerPacket', 'userMaxBytesPerPacket', 'userMedBytesPerPacket', 'userAvgBytesPerPacket', 'userStdBytesPerPacket', 'userKurBytesPerPacket', 'userSkeBytesPerPacket', 'userMinBytesInFlight', 'userMaxBytesInFlight', 'userMedBytesInFlight', 'userAvgBytesInFlight', 'userStdBytesInFlight', 'userKurBytesInFlight', 'userSkeBytesInFlight', 'userMinInterArrivalTime', 'userMaxInterArrivalTime', 'userMedInterArrivalTime', 'userAvgInterArrivalTime', 'userStdInterArrivalTime', 'userKurInterArrivalTime', 'userSkeInterArrivalTime', 'userMinRwnd', 'userMaxRwnd', 'userMedRwnd', 'userAvgRwnd', 'userStdRwnd', 'userKurRwnd', 'userSkeRwnd', 'userThroughput', 'userIdleTime', 'userOutOfOrderBytes', 'userOutOfOrderPackets', 'userZeroRetransmit', 'userOneRetransmit', 'userTwoRetransmit', 'userXRetransmit', 'userMinRetransmit', 'userMaxRetransmit', 'userMedRetransmit', 'userAvgRetransmit', 'userStdRetransmit', 'userKurRetransmit', 'userSkeRetransmit', 'userGoodput', 'userMinRTT', 'userMaxRTT', 'userMedRTT', 'userAvgRTT', 'userStdRTT', 'userKurRTT', 'userSkeRTT']

#started
#feature_columns = ['started', 'video', 'serverSynFlags', 'serverFinFlags', 'serverAckFlags', 'serverUrgFlags', 'serverRstFlags', 'serverPshFlags', 'serverByteCount', 'serverPacketCount', 'serverMinBytesPerPacket', 'serverMaxBytesPerPacket', 'serverMedBytesPerPacket', 'serverAvgBytesPerPacket', 'serverStdBytesPerPacket', 'serverKurBytesPerPacket', 'serverSkeBytesPerPacket', 'serverMinBytesInFlight', 'serverMaxBytesInFlight', 'serverMedBytesInFlight', 'serverAvgBytesInFlight', 'serverStdBytesInFlight', 'serverKurBytesInFlight', 'serverSkeBytesInFlight', 'serverMinInterArrivalTime', 'serverMaxInterArrivalTime', 'serverMedInterArrivalTime', 'serverAvgInterArrivalTime', 'serverStdInterArrivalTime', 'serverKurInterArrivalTime', 'serverSkeInterArrivalTime', 'serverMinRwnd', 'serverMaxRwnd', 'serverMedRwnd', 'serverAvgRwnd', 'serverStdRwnd', 'serverKurRwnd', 'serverSkeRwnd', 'serverThroughput', 'serverIdleTime', 'serverOutOfOrderBytes', 'serverOutOfOrderPackets', 'serverZeroRetransmit', 'serverOneRetransmit', 'serverTwoRetransmit', 'serverXRetransmit', 'serverMinRetransmit', 'serverMaxRetransmit', 'serverMedRetransmit', 'serverAvgRetransmit', 'serverStdRetransmit', 'serverKurRetransmit', 'serverSkeRetransmit', 'serverGoodput', 'userSynFlags', 'userFinFlags', 'userAckFlags', 'userUrgFlags', 'userRstFlags', 'userPshFlags', 'userByteCount', 'userPacketCount', 'userMinBytesPerPacket', 'userMaxBytesPerPacket', 'userMedBytesPerPacket', 'userAvgBytesPerPacket', 'userStdBytesPerPacket', 'userKurBytesPerPacket', 'userSkeBytesPerPacket', 'userMinBytesInFlight', 'userMaxBytesInFlight', 'userMedBytesInFlight', 'userAvgBytesInFlight', 'userStdBytesInFlight', 'userKurBytesInFlight', 'userSkeBytesInFlight', 'userMinInterArrivalTime', 'userMaxInterArrivalTime', 'userMedInterArrivalTime', 'userAvgInterArrivalTime', 'userStdInterArrivalTime', 'userKurInterArrivalTime', 'userSkeInterArrivalTime', 'userMinRwnd', 'userMaxRwnd', 'userMedRwnd', 'userAvgRwnd', 'userStdRwnd', 'userKurRwnd', 'userSkeRwnd', 'userThroughput', 'userIdleTime', 'userOutOfOrderBytes', 'userOutOfOrderPackets', 'userZeroRetransmit', 'userOneRetransmit', 'userTwoRetransmit', 'userXRetransmit', 'userMinRetransmit', 'userMaxRetransmit', 'userMedRetransmit', 'userAvgRetransmit', 'userStdRetransmit', 'userKurRetransmit', 'userSkeRetransmit', 'userGoodput', 'userMinRTT', 'userMaxRTT', 'userMedRTT', 'userAvgRTT', 'userStdRTT', 'userKurRTT', 'userSkeRTT']

print(feature_columns)

['video', 'serverSynFlags', 'serverFinFlags', 'serverAckFlags', 'serverUrgFlags', 'serverRstFlags', 'serverPshFlags', 'serverByteCount', 'serverPacketCount', 'serverMinBytesPerPacket', 'serverMaxBytesPerPacket', 'serverMedBytesPerPacket', 'serverAvgBytesPerPacket', 'serverStdBytesPerPacket', 'serverKurBytesPerPacket', 'serverSkeBytesPerPacket', 'serverMinBytesInFlight', 'serverMaxBytesInFlight', 'serverMedBytesInFlight', 'serverAvgBytesInFlight', 'serverStdBytesInFlight', 'serverKurBytesInFlight', 'serverSkeBytesInFlight', 'serverMinInterArrivalTime', 'serverMaxInterArrivalTime', 'serverMedInterArrivalTime', 'serverAvgInterArrivalTime', 'serverStdInterArrivalTime', 'serverKurInterArrivalTime', 'serverSkeInterArrivalTime', 'serverMinRwnd', 'serverMaxRwnd', 'serverMedRwnd', 'serverAvgRwnd', 'serverStdRwnd', 'serverKurRwnd', 'serverSkeRwnd', 'serverThroughput', 'serverIdleTime', 'serverOutOfOrderBytes', 'serverOutOfOrderPackets', 'serverZeroRetransmit', 'serverOneRetransmit', 'serverTwoRe

In [23]:
def make_input_fn(data_df, label_df, num_epochs=10, shuffle=True, batch_size=256):
  def input_function():  # inner function, this will be returned
    ds = tf.data.Dataset.from_tensor_slices((dict(data_df), label_df))  # create tf.data.Dataset object with data and its label
    if shuffle:
      ds = ds.shuffle(1000)  # randomize order of data
    ds = ds.batch(batch_size).repeat(num_epochs)  # split dataset into batches of 32 and repeat process for number of epochs
    return ds  # return a batch of the dataset
  return input_function  # return a function object for use

train_input_fn = make_input_fn(dftrain, y_train)  # here we will call the input_function that was returned to us to get a dataset object we can feed to the model
eval_input_fn = make_input_fn(dfeval, y_eval, num_epochs=1, shuffle=False)



In [24]:
linear_est = tf.estimator.LinearClassifier(feature_columns=feature_columns)
# We create a linear estimtor by passing the feature columns we created earlier

linear_est.train(train_input_fn)  # train
result = linear_est.evaluate(eval_input_fn)  # get model metrics/stats by testing on tetsing data

clear_output()  # clears consoke output
print(result['accuracy'])  # the result variable is simply a dict of stats about our model

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': 'C:\\Users\\tisa_\\AppData\\Local\\Temp\\tmps4dywlf7', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_session_creation_timeout_secs': 7200, '_service': None, '_cluster_spec': ClusterSpec({}), '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}
INFO:tensorflow:Calling model_fn.


ValueError: Received a feature column from TensorFlow v1, but this is a TensorFlow v2 Estimator. Please either use v2 feature columns (accessible via tf.feature_column.* in TF 2.x) with this Estimator, or switch to a v1 Estimator for use with v1 feature columns (accessible via tf.compat.v1.estimator.* and tf.compat.v1.feature_column.*, respectively.

In [25]:
def input_fn(features, labels, training=True, batch_size=256):
    # Convert the inputs to a Dataset.
    dataset = tf.data.Dataset.from_tensor_slices((dict(features), labels))

    # Shuffle and repeat if you are in training mode.
    if training:
        dataset = dataset.shuffle(1).repeat()
    
    return dataset.batch(batch_size)


In [26]:
# Feature columns describe how to use the input.
my_feature_columns = []
for key in dftrain.keys():
    my_feature_columns.append(tf.feature_column.numeric_column(key=key))
print(my_feature_columns)

[NumericColumn(key='serverSynFlags', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None), NumericColumn(key='serverFinFlags', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None), NumericColumn(key='serverAckFlags', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None), NumericColumn(key='serverUrgFlags', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None), NumericColumn(key='serverRstFlags', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None), NumericColumn(key='serverPshFlags', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None), NumericColumn(key='serverByteCount', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None), NumericColumn(key='serverPacketCount', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None), NumericColumn(key='serverMinBytesPerPacket', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None), NumericColumn(key='serv

In [28]:
classifier = tf.estimator.DNNClassifier(
    feature_columns=my_feature_columns,
    hidden_units=[30, 10, 20, 25, 35],
    n_classes=2)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': 'C:\\Users\\tisa_\\AppData\\Local\\Temp\\tmp9ni1hsn3', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_session_creation_timeout_secs': 7200, '_service': None, '_cluster_spec': ClusterSpec({}), '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


In [29]:
classifier.train(
    input_fn=lambda: input_fn(dftrain, y_train, training=True),
    steps=6000) 


INFO:tensorflow:Calling model_fn.

If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.

INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Calling checkpoint listeners before saving checkpoint 0...
INFO:tensorflow:Saving checkpoints for 0 into C:\Users\tisa_\AppData\Local\Temp\tmp9ni1hsn3\model.ckpt.
INFO:tensorflow:Calling checkpoint listeners after saving checkpoint 0...
INFO:tensorflow:lo

In [30]:
eval_result = classifier.evaluate(
    input_fn=lambda: input_fn(dfeval, y_eval, training=False))

print('\nTest set accuracy: {accuracy:0.3f}\n'.format(**eval_result))

INFO:tensorflow:Calling model_fn.

If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.

INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2020-09-26T22:28:31Z
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from C:\Users\tisa_\AppData\Local\Temp\tmp9ni1hsn3\model.ckpt-6000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Inference Time : 10.68513s
INFO:tensorflow:Finished evaluation at 2020-09-26-22:28:42
INFO:tensorflow:Saving dict for global step 6000: